In [1]:
!pip install transformers==4.20.0
!pip install keras_nlp==0.3.0
!pip install rouge-score
!pip install git+https://github.com/google-research/bleurt.git
!pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-e5v2r6jm
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-e5v2r6jm
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import transformers
from bleurt import score
from tensorflow import keras
import keras_nlp
import pandas as pd
from datasets import load_dataset

In [16]:
t5_tuned = transformers.TFAutoModelForSeq2SeqLM.from_pretrained('model', local_files_only=True)
t5 = transformers.TFAutoModelForSeq2SeqLM.from_pretrained('t5-small')

tokenizer = transformers.AutoTokenizer.from_pretrained('t5-small')

pipe_t5_tuned = transformers.pipeline('summarization', tokenizer=tokenizer, model=t5_tuned, framework='tf')
pipe_t5 = transformers.pipeline('summarization', tokenizer=tokenizer, model=t5, framework='tf')


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwar

In [7]:
rouge_l_metric = keras_nlp.metrics.RougeL()
bleurt_metric = score.create_bleurt_ops()

In [8]:
test_df = pd.read_csv('sub_val.csv')

In [9]:
prefix = "summarize: "
test_df['article'] = prefix + test_df['article']

In [10]:
test_df.iloc[:10]['article']

0    summarize: Paul McCartney's rich neighbours ha...
1    summarize: Petty: Eldridge Dukes, 56, is facin...
2    summarize: Rafael Nadal eased into the third r...
3    summarize: A jeering mob encouraged a suicidal...
4    summarize: Arsene Wenger is set to receive a h...
5    summarize: A lineman who suits up for the Balt...
6    summarize: The University of Texas is investig...
7    summarize: For Poldark’s legion of breathless ...
8    summarize: Swansea manager Garry Monk feels hi...
9    summarize: AP McCoy on Thursday secured his fi...
Name: article, dtype: object

### ROUGE-L metric on the pretrained t5 model


In [11]:
outputs = pipe_t5(test_df.iloc[:3]['article'].to_list())

Token indices sequence length is longer than the specified maximum sequence length for this model (1357 > 512). Running this sequence through the model will result in indexing errors


In [12]:
outputs = [output['summary_text'] for output in outputs]

In [13]:
rouge_l_metric(test_df.iloc[:3]['highlights'].to_list(), outputs)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


{'precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.23569024>,
 'recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.21517469>,
 'f1_score': <tf.Tensor: shape=(), dtype=float32, numpy=0.22428311>}

In [15]:
bleurt_metric(references=test_df.iloc[:3]['highlights'].to_list(), candidates=outputs)

{'predictions': <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.30918932, -0.33625358, -0.4281149 ], dtype=float32)>}

### ROUGE-L metric on the finetuned t5 model


In [17]:
outputs = pipe_t5_tuned(test_df.iloc[:3]['article'].to_list())
outputs = [output['summary_text'] for output in outputs]
rouge_l_metric(test_df.iloc[:3]['highlights'].to_list(), outputs)

Token indices sequence length is longer than the specified maximum sequence length for this model (1357 > 512). Running this sequence through the model will result in indexing errors


{'precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.2585937>,
 'recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.20762108>,
 'f1_score': <tf.Tensor: shape=(), dtype=float32, numpy=0.22665435>}

In [19]:
bleurt_metric(references=test_df.iloc[:3]['highlights'].to_list(), candidates=outputs)


{'predictions': <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.622548  , -0.51502115, -0.51887476], dtype=float32)>}